In [1]:
# Import pandas dependency
import pandas as pd
import timeit
import numpy as np
import datetime
from sqlalchemy import create_engine

In [2]:
#Read HGAC all county crash data into dataframe
all_years_data_df = pd.read_csv("Resources/Crash_Data/Processed/AllTravisCrashData.csv")


C:\Users\GHANDOURH\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3072: DtypeWarning: Columns (19,20,22,34,35,37,46,84,154,155) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
#Clip data on on-system highways only
all_years_data_df = all_years_data_df[all_years_data_df["Onsys_Fl"]=="Y"]

In [4]:
#Exporting unique street names
df=pd.DataFrame(all_years_data_df.Street_Name.unique())
df.to_csv("Resources/Street_Names/unique_Street_Name.csv")

In [5]:
#Create column "Year"
all_years_data_df["Year"] =pd.to_datetime(all_years_data_df["Crash_Date"]).dt.strftime('%Y')

In [6]:
#Store Prefix for all highways
AllHighwaysPrefix = ["SH","IH","SS","UA","FM","US","BS","SL","BU","FS","BF"]

In [7]:
#Drop NOT needed columns in Crashes Data
#all_years_data_df.columns.to_list()

In [8]:
#Create subset of data that will be needed for analysis
all_years_data_df = all_years_data_df[["Crash_ID", "Cmv_Involv_Fl", "Crash_Date", "Crash_Time", 
                             "Crash_Speed_Limit", "Wthr_Cond_ID", "Light_Cond_ID", 
                             "Surf_Cond_ID", "Traffic_Cntl_ID", "Harm_Evnt_ID", "Intrsct_Relat_ID",
                             "FHE_Collsn_ID", "Road_Relat_ID", "Cnty_ID", "City_ID", "Latitude",
                             "Longitude", "Hwy_Sys", "Hwy_Nbr", "Dfo", "Street_Name", "Onsys_Fl",
                             "Rural_Fl", "Crash_Sev_ID", "Day_of_Week", "Shldr_Width_Left",
                             "Shldr_Width_Right", "Median_Width", "Nbr_Of_Lane", "Func_Sys_ID",
                             "Adt_Curnt_Amt", "Adt_Curnt_Year", "Trk_Aadt_Pct", "Curve_Type_ID",
                             "Curve_Lngth", "Cd_Degr", "Sus_Serious_Injry_Cnt", "Nonincap_Injry_Cnt",
                             "Poss_Injry_Cnt", "Non_Injry_Cnt", "Unkn_Injry_Cnt", "Tot_Injry_Cnt", "Death_Cnt", "Year"]]
all_years_data_df.head()


,Crash_ID,Cmv_Involv_Fl,Crash_Date,Crash_Time,Crash_Speed_Limit,Wthr_Cond_ID,Light_Cond_ID,Surf_Cond_ID,Traffic_Cntl_ID,Harm_Evnt_ID,...,Curve_Lngth,Cd_Degr,Sus_Serious_Injry_Cnt,Nonincap_Injry_Cnt,Poss_Injry_Cnt,Non_Injry_Cnt,Unkn_Injry_Cnt,Tot_Injry_Cnt,Death_Cnt,Year
3,14839123,N,01/01/2016,12:23 AM,80,11,4,1,20,10,...,1686.0,0.0,0,1,0,0,0,1,0,2016
5,14839307,N,01/01/2016,07:59 PM,60,12,3,1,5,2,...,303.0,4.0,0,0,0,2,0,0,0,2016
7,14839787,N,01/01/2016,07:40 PM,-1,12,4,1,5,8,...,NaN,NaN,0,0,0,2,0,0,0,2016
8,14839805,N,01/02/2016,03:44 PM,45,2,1,2,5,2,...,179.0,0.0,0,0,0,2,0,0,0,2016
9,14839874,N,01/01/2016,05:11 PM,45,12,6,1,11,2,...,NaN,NaN,0,0,1,4,0,1,0,2016


In [9]:
#Create crash output for onsystem crashes
all_years_data_df.to_csv("Resources/Crash_Data/Processed/OnSystemTravisCrashes.csv",index=False)

In [10]:
#Create and export only severe (K+A+B) crashes
severe_data_df = all_years_data_df[(all_years_data_df["Crash_Sev_ID"]==1) |
                                  (all_years_data_df["Crash_Sev_ID"]==2) |
                                  (all_years_data_df["Crash_Sev_ID"]==4)]
severe_data_df.to_csv("Resources/Crash_Data/Processed/SevereOnSystemTravisCrashes.csv",index=False)

In [11]:
#View data type
#all_years_data_df.dtypes

In [12]:
#Read RINO File
All_RINO_df=pd.read_csv("Resources/Roadway_Data/TravisOnSystemRINO.csv")

In [13]:
#All_RINO_df.columns.to_list()

In [14]:
#Create Subset of Roadway data
All_RINO_df = All_RINO_df[["REC", "RIA_RTE_ID", "FRM_DFO", "TO_DFO", "HWY", "HSYS",
                           "HNUM", "DI", "CO", "CITY", "RU", "F_SYSTEM", "SPD_MAX",
                           "HP_MED_W", "NUM_LANES", "SUR_W", "ADT_YEAR", "ADT_CUR", 
                           "ADT_ADJ", "TRK_AADT_PCT", "AADT_TRUCKS", "LANE_WIDTH", "LEN_SEC",
                           "LN_MILES", "DVMT", "DTRKVMT","HY_1","HY_2","HY_3","HY_4"]]
All_RINO_df

,REC,RIA_RTE_ID,FRM_DFO,TO_DFO,HWY,HSYS,HNUM,DI,CO,CITY,...,AADT_TRUCKS,LANE_WIDTH,LEN_SEC,LN_MILES,DVMT,DTRKVMT,HY_1,HY_2,HY_3,HY_4
0,1,FM0685-KG,7.856,7.894,FM0685,FM,685,14,227,32580,...,1288,12,0.038,0.152,1129.892,48.944,29334,26538,28012,24253
1,1,FM0685-KG,5.858,6.348,FM0685,FM,685,14,227,32580,...,1288,13,0.490,1.960,14569.660,631.120,29334,26538,28012,24253
2,1,FM0685-KG,7.894,7.958,FM0685,FM,685,14,227,32580,...,1288,12,0.064,0.256,1902.976,82.432,29334,26538,28012,24253
3,1,FM0685-KG,5.599,5.858,FM0685,FM,685,14,227,32580,...,1288,12,0.259,1.036,7701.106,333.592,29334,26538,28012,24253
4,1,FM0685-KG,6.367,7.856,FM0685,FM,685,14,227,32580,...,1288,13,1.489,5.956,44273.926,1917.832,29334,26538,28012,24253
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1824,1,US0290-KG,135.429,135.456,US0290,US,290,14,227,26450,...,3083,12,0.027,0.108,1206.927,83.241,44701,36907,38829,33894
1825,1,US0290-KG,103.649,103.994,US0290,US,290,14,227,0,...,1106,12,0.345,1.380,12999.600,381.570,37680,36025,38369,31294
1826,1,US0290-KG,132.639,132.901,US0290,US,290,14,227,2100,...,1981,12,0.262,1.048,4971.450,519.022,18975,16148,15597,11263
1827,1,US0290-KG,111.507,112.341,US0290,US,290,14,227,2100,...,1809,12,0.834,4.170,57022.248,1508.706,68372,65855,65855,64393


In [15]:
#All_RINO_df.dtypes

In [16]:
#Write adjusted RINO Data
All_RINO_df.to_csv("Resources/Roadway_Data/TravisOnSystemRINOAdjusted.csv",index=False)

In [17]:
#Sort data by HWY name and DFO
All_RINO_df = All_RINO_df.sort_values(by=['HWY',"FRM_DFO"], ascending=(True,True))
#Add Corridor ID column for each unique street name
All_RINO_df["Corridor_ID"]=1
#Add Roadway ID column for each roadway segment
All_RINO_df["Road_ID"]=1

In [18]:
#Showing unique street names
unique_streets=All_RINO_df.HWY.unique()
unique_streets

array(['FM0685', 'FM0734', 'FM0812', 'FM0969', 'FM0973', 'FM1100',
       'FM1325', 'FM1327', 'FM1625', 'FM1626', 'FM1825', 'FM2304',
       'FM3177', 'FS1825', 'IH0035', 'RM0012', 'RM0620', 'RM1431',
       'RM1826', 'RM2222', 'RM2244', 'RM2322', 'RM2769', 'RM3238',
       'SH0045', 'SH0071', 'SH0095', 'SH0130', 'SH0165', 'SL0001',
       'SL0111', 'SL0212', 'SL0275', 'SL0343', 'SL0360', 'SS0069',
       'US0183', 'US0290'], dtype=object)

In [19]:
#Adjust Corridor ID numbering to be unique for each corridor name
for i in range(1, len(All_RINO_df) ):
    
    j = All_RINO_df.columns.get_loc('Corridor_ID')
    k = All_RINO_df.columns.get_loc('HWY')
    l = All_RINO_df.columns.get_loc('Road_ID')
    
    All_RINO_df.iat[i , l] = All_RINO_df.iat[i - 1, l] + 1
    
    if All_RINO_df.iat[i , k]==All_RINO_df.iat[i - 1, k]:
        All_RINO_df.iat[i , j] = All_RINO_df.iat[i - 1, j]
        
    else:
        All_RINO_df.iat[i , j] = All_RINO_df.iat[i - 1, j] + 1

In [20]:
#Get Boundary DFO
Min_DFO_Limit=All_RINO_df.groupby("Corridor_ID")["FRM_DFO"].min()
Max_DFO_Limit=All_RINO_df.groupby("Corridor_ID")["TO_DFO"].max()
#Get Corridor Name
Corridor_Names=All_RINO_df.groupby("Corridor_ID")["HWY"].max()
#Calculate Corridor Length
Corridor_Lengths=All_RINO_df.groupby("Corridor_ID")["LEN_SEC"].sum()

In [21]:
#Create Dataframe with Corridor Names, Numbers, and Min and Max DFO
Corridor_Summary_df=pd.DataFrame({"Highway Name":Corridor_Names,"From DFO":Min_DFO_Limit,"To DFO":Max_DFO_Limit,"Length (mi)":Corridor_Lengths})
Corridor_Summary_df

,Highway Name,From DFO,To DFO,Length (mi)
Corridor_ID,,,,
1,FM0685,5.599,7.958,2.359
2,FM0734,6.547,19.385,12.838
3,FM0812,0.000,6.979,6.979
4,FM0969,0.000,16.712,16.712
5,FM0973,6.809,36.403,27.603
6,FM1100,0.000,6.520,6.520
7,FM1325,1.448,8.171,3.484
8,FM1327,0.000,7.331,7.331
9,FM1625,0.000,5.130,5.130


In [22]:
#Join Crashes and RINO data
join_df = pd.merge(all_years_data_df,All_RINO_df, how='outer',left_on='Street_Name',right_on='HWY')

#Condition join for only crashes with DFO within limits
join_df_inner= join_df[(join_df["Dfo"]>=join_df["FRM_DFO"]) &(join_df["Dfo"]<join_df["TO_DFO"])]

In [23]:
join_df_inner

,Crash_ID,Cmv_Involv_Fl,Crash_Date,Crash_Time,Crash_Speed_Limit,Wthr_Cond_ID,Light_Cond_ID,Surf_Cond_ID,Traffic_Cntl_ID,Harm_Evnt_ID,...,LEN_SEC,LN_MILES,DVMT,DTRKVMT,HY_1,HY_2,HY_3,HY_4,Corridor_ID,Road_ID
21,14839123,N,01/01/2016,12:23 AM,80,11,4,1,20,10,...,2.350,9.400,107916.700,22407.250,43413.0,43788.0,42315.0,38748.0,28.0,1188.0
83,14840287,N,01/01/2016,07:40 AM,80,11,1,1,11,7,...,2.642,10.568,121325.924,25191.470,43413.0,43788.0,42315.0,38748.0,28.0,1192.0
143,14840860,N,01/03/2016,06:48 PM,80,11,4,1,1,6,...,1.213,4.852,55703.386,11565.955,43413.0,43788.0,42315.0,38748.0,28.0,1194.0
214,14850383,N,01/07/2016,09:05 AM,80,5,1,1,20,2,...,0.099,0.396,3340.359,796.257,31038.0,30351.0,28878.0,25311.0,28.0,1207.0
245,14863190,N,01/12/2016,04:01 PM,65,11,1,1,5,2,...,0.036,0.144,1508.940,133.344,39424.0,41579.0,39927.0,36360.0,28.0,1180.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5887310,17293867,N,09/06/2019,11:43 AM,45,11,1,1,8,2,...,0.527,1.054,5505.042,349.928,10460.0,10329.0,9326.0,7479.0,14.0,489.0
5887315,17302636,N,09/22/2019,12:40 PM,45,11,1,1,11,2,...,0.527,1.054,5505.042,349.928,10460.0,10329.0,9326.0,7479.0,14.0,489.0
5887323,17318171,N,09/30/2019,10:45 AM,45,2,1,2,17,2,...,0.119,0.238,1243.074,79.016,10460.0,10329.0,9326.0,7479.0,14.0,492.0
5887329,17361764,N,10/24/2019,02:07 PM,45,12,1,1,20,2,...,0.037,0.074,386.502,24.568,10460.0,10329.0,9326.0,7479.0,14.0,493.0


In [24]:
#Create variable to calculate Average AADT
Average_AADT = (join_df_inner["ADT_ADJ"]+join_df_inner["HY_1"]+join_df_inner["HY_2"]
                +join_df_inner["HY_3"]+join_df_inner["HY_4"])/5

#Add column that computes average AADT
join_df_inner["Average_AADT"] =Average_AADT.to_list()

C:\Users\GHANDOURH\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [25]:
#Add RLD, PED, & INT columns
join_df_inner["RLD"]=0
join_df_inner["PED"]=0
join_df_inner["INT"]=0
#Adjust new column data
for i in range(0, len(join_df_inner)):
    
    j = join_df_inner.columns.get_loc('RLD')
    k = join_df_inner.columns.get_loc('PED')
    l = join_df_inner.columns.get_loc('INT')
    m = join_df_inner.columns.get_loc('Intrsct_Relat_ID')
    n = join_df_inner.columns.get_loc('FHE_Collsn_ID')
    o = join_df_inner.columns.get_loc('Harm_Evnt_ID')
    
    if join_df_inner.iat[i, m]<4:
        join_df_inner.iat[i, l] = 1
        
    if join_df_inner.iat[i, o]==1:
        join_df_inner.iat[i, k] = 1
        
    if join_df_inner.iat[i, n]==30:
        join_df_inner.iat[i, j] = 1
        
    if (join_df_inner.iat[i, n]<5) & (join_df_inner.iat[i, o]>6):
        join_df_inner.iat[i, j] = 1

C:\Users\GHANDOURH\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\GHANDOURH\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\GHANDOURH\Anaconda3\envs\PythonData\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [26]:
#Append All Crashes that meet the Highway Name and DFO Criteria
#tempdf= pd.DataFrame()
#Top10CongestedCrashes_df= pd.DataFrame()
#for i in Top10_Summary_df.index:
    #tempdf=all_years_data_df[(all_years_data_df["Street_Name"]==Top10_Summary_df["Highway Name"][i])&(all_years_data_df["Dfo"]>Top10_Summary_df["From DFO"][i])
                         #&(all_years_data_df["Dfo"]<Top10_Summary_df["To DFO"][i])]
    #Top10CongestedCrashes_df=Top10CongestedCrashes_df.append(tempdf)


In [27]:
#Create a seperate Df for each of the top 10 congested corridors
#corridor1_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[0]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[0]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[0]["To DFO"])]
#corridor1_Crashes_df.loc[:,'Top10']=1
#corridor2_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[1]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[1]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[1]["To DFO"])]
#corridor2_Crashes_df.loc[:,'Top10']=2

#corridor3_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[2]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[2]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[2]["To DFO"])]
#corridor3_Crashes_df.loc[:,'Top10']=3

#corridor4_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[3]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[3]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[3]["To DFO"])]
#corridor4_Crashes_df.loc[:,'Top10']=4

#corridor5_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[4]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[4]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[4]["To DFO"])]
#corridor5_Crashes_df.loc[:,'Top10']=5

#corridor6_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[5]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[5]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[5]["To DFO"])]
#corridor6_Crashes_df.loc[:,'Top10']=6

#corridor7_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[6]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[6]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[6]["To DFO"])]
#corridor7_Crashes_df.loc[:,'Top10']=7

#corridor8_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[7]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[7]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[7]["To DFO"])]
#corridor8_Crashes_df.loc[:,'Top10']=8

#corridor9_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[8]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[8]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[8]["To DFO"])]
#corridor9_Crashes_df.loc[:,'Top10']=9

#corridor10_Crashes_df=Top10CongestedCrashes_df[(Top10CongestedCrashes_df["Street_Name"]==Top10_Summary_df.iloc[9]["Highway Name"])
                                              #&(Top10CongestedCrashes_df["Dfo"]>Top10_Summary_df.iloc[9]["From DFO"])
                                              #&(Top10CongestedCrashes_df["Dfo"]<Top10_Summary_df.iloc[9]["To DFO"])]
#corridor10_Crashes_df.loc[:,'Top10']=10

In [28]:
#Create one dataframe holding all crash data
#Top10CongestedCrashes_df=pd.concat([corridor1_Crashes_df,corridor2_Crashes_df,corridor3_Crashes_df,corridor4_Crashes_df,
                                   #corridor5_Crashes_df,corridor6_Crashes_df,corridor7_Crashes_df,corridor8_Crashes_df,
                                   #corridor9_Crashes_df,corridor10_Crashes_df])
#Save File
#Top10CongestedCrashes_df.to_csv("Resources/Crash_Data/Processed/FortBendCrashesAdjusted.csv")

In [29]:
#Create number of crashes list by corridor
Corridor_num_crashes=join_df_inner.groupby(["Corridor_ID"])["Crash_ID"].count()

In [30]:
#Summarize number of crashes by corridor
Corridor_Summary_df["Number of Crashes (5 Years)"]=Corridor_num_crashes

In [31]:
#Summarize number of crashes by mile by corridor
Corridor_Summary_df["Crashes Per Mile Per Year"]=Corridor_Summary_df["Number of Crashes (5 Years)"]/(Corridor_Summary_df["Length (mi)"]*5)

In [32]:
#Show summary
Corridor_Summary_df

,Highway Name,From DFO,To DFO,Length (mi),Number of Crashes (5 Years),Crashes Per Mile Per Year
Corridor_ID,,,,,,
1,FM0685,5.599,7.958,2.359,548,46.460365
2,FM0734,6.547,19.385,12.838,1579,24.598847
3,FM0812,0.000,6.979,6.979,724,20.747958
4,FM0969,0.000,16.712,16.712,1148,13.738631
5,FM0973,6.809,36.403,27.603,850,6.158751
6,FM1100,0.000,6.520,6.520,30,0.920245
7,FM1325,1.448,8.171,3.484,495,28.415614
8,FM1327,0.000,7.331,7.331,168,4.583276
9,FM1625,0.000,5.130,5.130,92,3.586745


In [33]:
#Summarize average,min, max number of lanes and average number of lanes for each corridor
Avrg_Lanes=join_df_inner.groupby("Corridor_ID")["NUM_LANES"].mean()
Min_Lanes=join_df_inner.groupby("Corridor_ID")["NUM_LANES"].min()
Max_Lanes=join_df_inner.groupby("Corridor_ID")["NUM_LANES"].max()

In [34]:
#Summarize average,min, max number of lanes and average number AADT for each corridor
Avrg_AADT=join_df_inner.groupby("Corridor_ID")["Average_AADT"].mean()
Min_AADT=join_df_inner.groupby("Corridor_ID")["Average_AADT"].min()
Max_AADT=join_df_inner.groupby("Corridor_ID")["Average_AADT"].max()

In [35]:
#Summarize average number of lanes Crashes Per Mile Per Lane
Corridor_Summary_df["Average Lanes"]=Avrg_Lanes
Corridor_Summary_df["Crashes Per Mile Per Lane Per Year"]=Corridor_Summary_df["Crashes Per Mile Per Year"]/(Corridor_Summary_df["Average Lanes"]*5)

In [36]:
#Summarize average AADT,VMT, and Crashes Per 100MVMT
Corridor_Summary_df["Average AADT"]=Avrg_AADT
Corridor_Summary_df["Average VMT (100M)"]=Corridor_Summary_df["Average AADT"]*365*Corridor_Summary_df["Length (mi)"]/100000000

In [37]:
Corridor_Summary_df

,Highway Name,From DFO,To DFO,Length (mi),Number of Crashes (5 Years),Crashes Per Mile Per Year,Average Lanes,Crashes Per Mile Per Lane Per Year,Average AADT,Average VMT (100M)
Corridor_ID,,,,,,,,,,
1,FM0685,5.599,7.958,2.359,548,46.460365,4.000000,2.323018,27574.200000,0.237424
2,FM0734,6.547,19.385,12.838,1579,24.598847,5.455351,0.901824,46547.160228,2.181139
3,FM0812,0.000,6.979,6.979,724,20.747958,2.696133,1.539090,15013.012707,0.382432
4,FM0969,0.000,16.712,16.712,1148,13.738631,3.317944,0.828141,20454.070557,1.247674
5,FM0973,6.809,36.403,27.603,850,6.158751,2.204706,0.558691,12537.315059,1.263146
6,FM1100,0.000,6.520,6.520,30,0.920245,2.000000,0.092025,1571.360000,0.037395
7,FM1325,1.448,8.171,3.484,495,28.415614,4.072727,1.395410,39193.740606,0.498411
8,FM1327,0.000,7.331,7.331,168,4.583276,2.023810,0.452936,13426.842857,0.359277
9,FM1625,0.000,5.130,5.130,92,3.586745,2.000000,0.358674,5704.582609,0.106815


In [38]:
#Calculate Crash Rate as Number of crashes by 100M VMT
Corridor_Summary_df["Crashes Per 100M VMT"]=Corridor_Summary_df["Number of Crashes (5 Years)"]/(Corridor_Summary_df["Average VMT (100M)"]*5)

In [39]:
Corridor_Summary_df

,Highway Name,From DFO,To DFO,Length (mi),Number of Crashes (5 Years),Crashes Per Mile Per Year,Average Lanes,Crashes Per Mile Per Lane Per Year,Average AADT,Average VMT (100M),Crashes Per 100M VMT
Corridor_ID,,,,,,,,,,,
1,FM0685,5.599,7.958,2.359,548,46.460365,4.000000,2.323018,27574.200000,0.237424,461.622350
2,FM0734,6.547,19.385,12.838,1579,24.598847,5.455351,0.901824,46547.160228,2.181139,144.786710
3,FM0812,0.000,6.979,6.979,724,20.747958,2.696133,1.539090,15013.012707,0.382432,378.629673
4,FM0969,0.000,16.712,16.712,1148,13.738631,3.317944,0.828141,20454.070557,1.247674,184.022464
5,FM0973,6.809,36.403,27.603,850,6.158751,2.204706,0.558691,12537.315059,1.263146,134.584558
6,FM1100,0.000,6.520,6.520,30,0.920245,2.000000,0.092025,1571.360000,0.037395,160.448291
7,FM1325,1.448,8.171,3.484,495,28.415614,4.072727,1.395410,39193.740606,0.498411,198.631202
8,FM1327,0.000,7.331,7.331,168,4.583276,2.023810,0.452936,13426.842857,0.359277,93.521031
9,FM1625,0.000,5.130,5.130,92,3.586745,2.000000,0.358674,5704.582609,0.106815,172.259713


In [40]:
#Summarize crashe statistics by corridor
Corridor_Summary_df.to_csv("Resources/Crash_Data/Processed/CrashStatisticsSummary.csv")

In [41]:
#Summarize Total Number by Severity
df=join_df_inner.groupby(['Corridor_ID','Crash_Sev_ID']).count()["Crash_ID"].to_frame()
# Reset the index on the DataFrame. This is needed to use the 'pivot()' function.
df = df.reset_index()

In [42]:
#Pivot Table of Crash Severities and format and rename columns
df=df.pivot(index="Corridor_ID",columns="Crash_Sev_ID",values="Crash_ID")
df

Crash_Sev_ID,0,1,2,3,4,5
Corridor_ID,,,,,,
1.0,2.0,6.0,58.0,101.0,1.0,380.0
2.0,13.0,49.0,287.0,365.0,13.0,852.0
3.0,30.0,38.0,107.0,148.0,12.0,389.0
4.0,32.0,52.0,233.0,220.0,11.0,600.0
5.0,34.0,43.0,136.0,148.0,15.0,474.0
6.0,2.0,2.0,3.0,1.0,NaN,22.0
7.0,6.0,17.0,83.0,110.0,6.0,273.0
8.0,8.0,13.0,25.0,33.0,5.0,84.0
9.0,9.0,7.0,14.0,24.0,1.0,37.0


In [43]:
#Export Joined File
join_df_inner.to_csv("Resources/Crash_Data/Processed/Crash&RoadwayData.csv",index=False)